In [2]:
import os
import pandas as pd
import sys
import time
from pathlib import Path
import snscrape.modules.twitter as sntwitter
import itertools
# handle with care!
import warnings
warnings.filterwarnings('ignore')

In [3]:
TWEET_INFO = ['Datetime',
              'Tweet Id',
              'Text',
              'Username',
              'Location',
              'in_reply_to_user_id',
              #  'in_reply_to_status_id',
              'reply_count',
              'retweet_count',
              'favorite_count',
              'follower_count',
              'friends_count',
              'total_tweets',
              'lang']

def user_query_tweets(location, text_query, count_limit, since_date, until_date):
    tweetlist2 = []
    # print('from:'+username+' '+text_query+' since:'+since_date+' until:'+until_date)
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper(text_query+' near:"'+location+'" within:20km'+' since:'+since_date+' until:'+until_date).get_items()):
        if i > count_limit:
            break
        if tweet.content:
            tweetlist2.append([tweet.date,
                               tweet.id,
                               tweet.rawContent,
                               tweet.user.username,
                               tweet.user.location,
                               tweet.inReplyToUser,
                            #    tweet.inReplyToTweetId,
                               tweet.replyCount,
                               tweet.retweetCount,
                               tweet.likeCount,
                               tweet.user.followersCount,
                               tweet.user.friendsCount,
                               tweet.user.statusesCount,
                               tweet.lang])
    # Creating a dataframe from the tweets list above
    tweets_df2 = pd.DataFrame(tweetlist2, columns=TWEET_INFO)
    return tweets_df2


def user_query_tweets_to_csv(file_path,username_list,query_terms,count,since_date,until_date, lang):
    tweets_df=pd.DataFrame(list(), columns=TWEET_INFO)
    tweet_cnt = []
    for i,query in enumerate(query_terms):
        for ii,name in enumerate(username_list):
            data=user_query_tweets(name ,query,count,since_date,until_date, lang)
            if not data.empty:
                tweets_df=pd.concat([tweets_df, data], axis = 0)
                tweet_cnt.append([query, name, data.shape[0]])

    tweets_df.to_csv(file_path,index=False,encoding='utf-8')
    return tweets_df, tweet_cnt

In [4]:
since_date = '2020-01-03'
until_date = '2022-06-03'
# total tweets extracted for each user
upper = 1e8
lang = 'en'
kuwait = user_query_tweets(location="Kuwait City, Kuwait",
                  text_query= 'kafala OR كفالة', count_limit= upper, since_date = since_date, until_date = until_date)
print(kuwait.shape)

Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&include_ext_has_nft_avatar=1&include_ext_is_blue_verified=1&include_ext_verified_type=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_ext_limited_action_results=false&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_ext_collab_control=true&include_ext_views=true&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&include_ext_sensitive_media_warning=true&include_ext_trusted_friends_metadata=true&send_error_codes=true&simple_quoted_tweet=true&q=kafala+OR+%D9%83%D9%81%D8%A7%D9%84%D8%A9+near%3A%22Kuwait+City%2C+Kuwait%22+within%3A20km+since%3A2020-01-03+until%3A2022-06-03&tweet_search_mode=live&count=20&query_source=spell

ScraperException: 4 requests to https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&include_ext_has_nft_avatar=1&include_ext_is_blue_verified=1&include_ext_verified_type=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_ext_limited_action_results=false&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_ext_collab_control=true&include_ext_views=true&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&include_ext_sensitive_media_warning=true&include_ext_trusted_friends_metadata=true&send_error_codes=true&simple_quoted_tweet=true&q=kafala+OR+%D9%83%D9%81%D8%A7%D9%84%D8%A9+near%3A%22Kuwait+City%2C+Kuwait%22+within%3A20km+since%3A2020-01-03+until%3A2022-06-03&tweet_search_mode=live&count=20&que

In [7]:
kuwait.lang.value_counts()

ar     597
en       5
et       3
und      2
qme      2
Name: lang, dtype: int64

In [26]:
dubai = user_query_tweets(location = "United Arab Emirates", text_query= 'kafala OR كفالة', count_limit= upper, since_date = since_date, until_date = until_date)
print(dubai.shape)

(55, 13)


In [43]:
dubai.to_csv("./kafala_dubai.csv")
kuwait.to_csv("./kafala_kuwait.csv")

In [33]:
# Set the search query for the city and time period
city = 'kafala OR كفالة since:2020-01-03 until:2022-06-03'

# Use TwitterSearchScraper to retrieve the tweets for the city and time period
tweets = list(sntwitter.TwitterSearchScraper(city).get_items())

# Get the total number of tweets for the city and time period
tweet_count = len(tweets)

# Print the result
print(f"Total tweets from CITY, COUNTRY: {tweet_count}")

Stopping after 20 empty pages


Total tweets from CITY, COUNTRY: 11369


In [45]:
tweet_list = []
for i, tweet in enumerate(tweets):
        if tweet.content:
            tweet_list.append([tweet.date,
                               tweet.id,
                               tweet.rawContent,
                               tweet.user.username,
                               tweet.user.location,
                               tweet.inReplyToUser,
                            #    tweet.inReplyToTweetId,
                               tweet.replyCount,
                               tweet.retweetCount,
                               tweet.likeCount,
                               tweet.user.followersCount,
                               tweet.user.friendsCount,
                               tweet.user.statusesCount,
                               tweet.lang])
    # Creating a dataframe from the tweets list above
tweets_df = pd.DataFrame(tweet_list, columns=TWEET_INFO)

In [46]:
tweets_df

,Datetime,Tweet Id,Text,Username,Location,in_reply_to_user_id,reply_count,retweet_count,favorite_count,follower_count,friends_count,total_tweets,lang
0,2022-06-02 23:53:47+00:00,1532510821519065102,@w_met30 عليج كفالة ولا ؟,3wajel,"beirut, lebanon",https://twitter.com/w_met30,1,0,0,2975,191,46490,ar
1,2022-06-02 23:31:23+00:00,1532505187272863745,@sm_11324 محمول بس بدون كفاله,m20g8,+968,https://twitter.com/sm_11324,1,0,1,18634,9198,75758,ar
2,2022-06-02 23:23:04+00:00,1532503092494848001,@SDB_Care ارغب في الغاء كفاله بالرغم ان الطلب ...,aburana1421,المملكة العربية السعودية,None,1,0,0,19,14,109,ar
3,2022-06-02 23:09:59+00:00,1532499801421631502,@abcd13200 @Huamn16 @uoshrm_sa @YourJobs_KSA ا...,Goudaelgendy2,,https://twitter.com/abcd13200,0,0,0,7,68,25,ar
4,2022-06-02 23:04:59+00:00,1532498542950076433,@_Ma_ai @_Nenaa2 كفالة معالق,Pajamaa_,,https://twitter.com/_Ma_ai,1,0,1,293,419,5081,ar
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11364,2022-04-13 19:25:47+00:00,1514323986955522048,🇸🇦 مـﻧـــ الخاصــ 📩\nمساء الخير .. عندي عامل و...,Ask_Alriyadh1,الرياض,None,93,1,40,435017,20,506796,ar
11365,2022-04-13 19:15:43+00:00,1514321450135658499,@abcd13200 السلام عليكم اذا رخصه العمل منتهيه ...,hossam_hemed,"Riyadh, Kingdom of Saudi Arabi",None,0,0,0,44,352,824,ar
11366,2022-04-13 19:12:08+00:00,1514320548280549385,ساهم معنا في كفالة الحلقة الخامسة السهم ١٠٠ ر...,bayanorgsa,السعودية - نجران,None,0,2,1,2981,21,3626,ar
11367,2022-04-13 19:11:38+00:00,1514320425618010113,كفالة يتيم عن سنة كاملة\nبتبرع كريم من الاخت ف...,MawaddaRelief,فلسطين,None,0,0,1,56,690,87,ar


In [47]:
tweets_df.to_csv("./all_kafala.csv")

In [32]:
# Set the search query for the city and time period
city = 'kafala OR كفالة near:"Dubai, UAE" within:20km since:2020-01-03 until:2022-06-03'

# Use TwitterSearchScraper to retrieve the tweets for the city and time period
tweets2 = list(sntwitter.TwitterSearchScraper(city).get_items())

# Get the total number of tweets for the city and time period
tweet_count_2 = len(tweets2)

# Print the result
print(f"Total tweets from CITY, COUNTRY: {tweet_count_2}")

Total tweets from CITY, COUNTRY: 1


## Query keywords based on location

In [6]:
query_terms_ar = ['کورونا','كوفيد',
    'المغتربين', 'العمالة الوافدة','عامل وافد', 'العمال المهاجرين', 'العمال الوطنيين', 'العمال غير المواطنين' ,
 'مواطن', 'غير مواطن'  , 'مقيم' , 'ساكن', 'التطعيم ضد كورونا', 'لقاح كورونا', 'الحجر صحي', 'دخول المستشفى' ]

query_terms_en = ['Coronavirus', 'Covid-19', 'Vaccine', 'Immunization', 'expat worker', 'expats',
 'expat workers', 'migrant workers', 'national workers', 'non-national workers',
  'citizen', 'non-citizen', 'resident', 'dweller', 'Quarantine', 'hospitalize']

since_date = '2020-01-03'
until_date = '2022-06-03'
# total tweets extracted for each user

In [11]:
dubai = user_query_tweets(location="Dubai, United Arab Emirates",
                          text_query='کورونا OR كوفيد',
                          count_limit=1e6,
                          since_date=since_date,
                          until_date=until_date)
dubai.shape

Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&include_ext_has_nft_avatar=1&include_ext_is_blue_verified=1&include_ext_verified_type=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_ext_limited_action_results=false&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_ext_collab_control=true&include_ext_views=true&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&include_ext_sensitive_media_warning=true&include_ext_trusted_friends_metadata=true&send_error_codes=true&simple_quoted_tweet=true&q=%DA%A9%D9%88%D8%B1%D9%88%D9%86%D8%A7+OR+%D9%83%D9%88%D9%81%D9%8A%D8%AF+near%3A%22Dubai%2C+United+Arab+Emirates%22+within%3A20km+since%3A2020-01-03+until%3A2022-06-03&tweet_search

ScraperException: 4 requests to https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&include_ext_has_nft_avatar=1&include_ext_is_blue_verified=1&include_ext_verified_type=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_ext_limited_action_results=false&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_ext_collab_control=true&include_ext_views=true&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&include_ext_sensitive_media_warning=true&include_ext_trusted_friends_metadata=true&send_error_codes=true&simple_quoted_tweet=true&q=%DA%A9%D9%88%D8%B1%D9%88%D9%86%D8%A7+OR+%D9%83%D9%88%D9%81%D9%8A%D8%AF+near%3A%22Dubai%2C+United+Arab+Emirates%22+within%3A20km+since%3A2020-01-03+until%3A2022-06-

In [5]:
## read old extracted tweets
df = pd.read_csv("./dubai_inf_en.csv")
df

,Datetime,Tweet Id,Text,Username,in_reply_to_user_id,reply_count,retweet_count,favorite_count,follower_count,friends_count,total_tweets,lang
0,2022-06-02 17:20:36+00:00,1532411876105236489,Today's look https://t.co/xFC1dOB25c,mustafa_agha,NaN,13,4,125,4584879,111,160643,en
1,2022-06-02 08:55:14+00:00,1532284695681155076,Just posted a photo https://t.co/YJF7INT2c6,mustafa_agha,NaN,5,0,10,4584879,111,160643,en
2,2022-06-01 17:24:28+00:00,1532050458566615042,Today's look https://t.co/s8rn0GxrLf,mustafa_agha,NaN,14,4,80,4584879,111,160643,en
3,2022-05-30 17:27:26+00:00,1531326431179022338,Today's look https://t.co/Z0pPhYUVmx,mustafa_agha,NaN,14,2,87,4584879,111,160643,en
4,2022-05-26 17:20:39+00:00,1529875171456307201,Today's look https://t.co/Q6n2Sjvgle,mustafa_agha,NaN,8,5,84,4584879,111,160643,en
...,...,...,...,...,...,...,...,...,...,...,...,...
14816,2021-06-17 14:29:10+00:00,1405532977430372361,@cryptochics @Davincij15 Woohoo! 🙌🏽🥳 We learnt...,BloomerBrittany,https://twitter.com/cryptochics,0,3,6,1041,177,1871,en
14817,2021-06-11 06:05:32+00:00,1403231909593944066,Amazing news 👏🏼 @awlnsw https://t.co/1BnJlsaC1W,BloomerBrittany,NaN,0,1,2,1041,177,1871,en
14818,2021-06-10 14:37:45+00:00,1402998422605426688,Woohoo! Check out what @frankimrussell and I h...,BloomerBrittany,NaN,3,7,17,1041,177,1871,en
14819,2020-08-21 00:14:03+00:00,1296601422213992450,@morningshowon7 Hello! Will you be sharing the...,BloomerBrittany,NaN,0,0,0,1041,177,1871,en


In [10]:
cond = df['Text'].apply(lambda x: any([k in x for k in query_terms_en]))
df.loc[cond,:].to_csv("./dubai_inf_en_keywords.csv",index=False)